In [58]:
from ad_config_search.waymo_utils import frames_from_path, simplify_ids, simplify_fnames, get_frame_dict
from pathlib import Path
import pickle
import ray

In [114]:
ray.shutdown()

In [109]:
ray.init(redis_password="asdfakljdsflkdjfldkfjldkfj23383u48hjfhfhfhf")

2020-11-25 12:05:18,760	INFO resource_spec.py:231 -- Starting Ray with 52.93 GiB memory available for workers and up to 26.49 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-11-25 12:05:19,166	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '169.229.48.111',
 'raylet_ip_address': '169.229.48.111',
 'redis_address': '169.229.48.111:6379',
 'object_store_address': '/tmp/ray/session_2020-11-25_12-05-18_756815_29435/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-25_12-05-18_756815_29435/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-11-25_12-05-18_756815_29435'}

In [110]:
data_path = Path("/data/ges/waymo/training_0005/")

In [75]:
name_map = simplify_fnames(data_path)

### Rename from hash to a readable name

In [76]:
for path in data_path.iterdir():
    ext = path.name.split(".")[-1]
    if ext != "tfrecord":
        continue
    new_name = path.parent/(name_map[path.stem]+".tfrecord")
    path.rename(new_name)

### Generate pl files

In [111]:
@ray.remote
def prepare_pl(data_path):
    data_path = Path(data_path)
    frames = frames_from_path(data_path)
    id_map = simplify_ids(frames)
    dicts = [get_frame_dict(f, id_map, rbg=False)["FRONT"] for f in frames]
    for d in dicts:
        d["center_camera_feed"] = d["img"]
        del d['img']
    with open(data_path.parent/(data_path.stem+".pl"), 'wb') as f:
        pickle.dump(dicts, f)
    return 1

In [112]:
callbacks = []
for path in data_path.iterdir():
    ext = path.name.split(".")[-1]
    if ext != "tfrecord":
        continue
    callbacks.append(prepare_pl.remote(path))
ray.wait(callbacks)

(pid=11738) 2020-11-25 12:05:23.438857: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
(pid=11738) 2020-11-25 12:05:23.439079: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
(pid=11738) 2020-11-25 12:05:23.439091: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
(pid=11653) 2020-11-25 12:05:23.443248: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
(pid=11

([ObjectRef(7575bebcb06c25a7ffffffff010000c001000000)],
 [ObjectRef(f21dc89fadaee218ffffffff010000c001000000),
  ObjectRef(46553e94847e2b37ffffffff010000c001000000),
  ObjectRef(6087e1b1ed98a18affffffff010000c001000000),
  ObjectRef(01ad35894212eb6affffffff010000c001000000),
  ObjectRef(c2f66569a02c666cffffffff010000c001000000),
  ObjectRef(d7bcc9b4e43bda85ffffffff010000c001000000),
  ObjectRef(6818461f4a23329fffffffff010000c001000000),
  ObjectRef(06d17ebff4fc93e3ffffffff010000c001000000),
  ObjectRef(52560c3652f3285fffffffff010000c001000000),
  ObjectRef(443e4949d2779ff5ffffffff010000c001000000),
  ObjectRef(b2356f7380ee1cb7ffffffff010000c001000000),
  ObjectRef(5503b58b2d0cce65ffffffff010000c001000000),
  ObjectRef(623f7cd8e611e75dffffffff010000c001000000),
  ObjectRef(efa8bd43e6b37b97ffffffff010000c001000000),
  ObjectRef(2ba66db8c83e52cfffffffff010000c001000000),
  ObjectRef(4ff517a2730e7822ffffffff010000c001000000),
  ObjectRef(98840b760a174a58ffffffff010000c001000000),
  ObjectR

2020-11-25 12:06:44,326	ERROR worker.py:1074 -- Possible unhandled error from worker: ray::__main__.prepare_pl() (pid=11687, ip=169.229.48.111)
  File "python/ray/_raylet.pyx", line 440, in ray._raylet.execute_task
  File "/home/eyal/miniconda3/envs/waymo_env/lib/python3.7/site-packages/ray/memory_monitor.py", line 128, in raise_if_low_memory
    self.error_threshold))
ray.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node erdos3 is used (92.43 / 94.06 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
11655	3.65GiB	ray::__main__.prepare_pl()
11668	3.65GiB	ray::__main__.prepare_pl()
11659	3.55GiB	ray::__main__.prepare_pl()
11738	3.51GiB	ray::__main__.prepare_pl()
11661	3.45GiB	ray::__main__.prepare_pl()
11651	3.43GiB	ray::__main__.prepare_pl()
11643	3.41GiB	ray::__main__.prepare_pl()
11656	3.38GiB	ray::__main__.prepare_pl()
11664	3.32GiB	ray::__main__.prepare_pl()
11678	3.32GiB	ray::__main__.prepare_pl()

In addition, up to 0.01 GiB of shared memory is currentl